In [7]:
import pypdf
import fitz # PyMuPDF

In [8]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        # Usando pypdf para PDFs nativos
        reader = pypdf.PdfReader(pdf_path)
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text() + "\n"
    except Exception as e:
        print(f"Error con pypdf, intentando con PyMuPDF: {e}")
        try:
            # Usando PyMuPDF para más robustez o coordenadas
            doc = fitz.open(pdf_path)
            for page in doc:
                text += page.get_text("text") + "\n" # "text" para texto simple, "words" para coordenadas
            doc.close()
        except Exception as e:
            print(f"Error con PyMuPDF: {e}")
            # Si falla, podría ser un PDF escaneado
            # Aquí iría la lógica para OCR (ver siguiente sección)
    return text

In [9]:
texto = extract_text_from_pdf("Resolucion_UNO_2025-01-03 14_09_09.271.pdf")
print(texto)

incorrect startxref pointer(1)
parsing for Object Streams


JUZGADO PENAL UNIPERSONAL - SEDE NCPP EL AGUSTINO 
EXPEDIENTE : 00341-2024-1-3203-JR-PE-02 
JUEZ   : LOAYZA SANCHEZ RUTH KARINA 
ESPECIALISTA : CONDORI ACUÑA HERBERTH 
MINISTERIO PUBLICO : PRIMERA FISCALIA SEGUNDO DESPA CHO EL 
AGUSTINO  
REPRESENTANTE : SIGUAS ZEGARRA, MONICA DEL ROSARIO 
IMPUTADO : BRACAMONTE PEREZ, FREDDY SAMMY 
DELITO  : OMISIÓN DE ASISTENCIA FAMILIAR 
AGRAVIADO : BRACAMONTE SIGUAS, DYLAND CALEB 
 
RESOLUCIÓN NÚMERO UNO    
El Agustino, veintiséis de diciembre  
Del dos mil veinticuatro. - 
    
   AUTOS Y VISTOS : AVOCÁNDOSE  la suscrita conforme a 
lo dispuesto en la Resolución Administrativa N°152- 2022-P-CSJLE-PJ; y al 
oficio con ingreso número 202583-2024 Téngase por r ecibido el expediente 
principal proveniente del Segundo Juzgado de Invest igación Preparatoria 
de El Agustino, conteniendo la acusación presentada  por la Primera 
fiscalía provincial Penal Corporativa de El Agustin o – Segundo despacho 
de Investigación, y, ATENDIENDO :    
    
PRIMERO .- Q

In [10]:
import re
import spacy

In [11]:
# Cargar modelo de Spacy (ej. para español)
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    print("Descargando modelo 'es_core_news_sm' de Spacy...")
    spacy.cli.download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")


In [13]:
EXCLUDED_LEGAL_TERMS = {
    "AUTOS",
    "VISTOS",
    "CONSIDERANDO",
    "CONSIDERANDOS",
    "RESUELVE",
    "POR TANTO",
    "DÉJESE CONSTANCIA",
    "SE RESUELVE",
    "ORDENA",
    "DECRETO",
    "LEGISLATIVO",
    # Añadir más términos según sea necesario
}

def find_pii(text):
    found_pii = []

    # 1. Regex para patrones comunes
    # Teléfono (ejemplo simple, ajustar a formatos específicos de Perú)
    phone_pattern = r'\b(?:\+?51)?\s?9\d{8}\b' # Adaptado para números de 9 dígitos que empiezan con 9 en Perú, opcional +51
    for match in re.finditer(phone_pattern, text):
        found_pii.append({'type': 'PHONE', 'text': match.group(0), 'start': match.start(), 'end': match.end()})

    # Email
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    for match in re.finditer(email_pattern, text):
        found_pii.append({'type': 'EMAIL', 'text': match.group(0), 'start': match.start(), 'end': match.end()})

    # DNI/CE (Ejemplo para Perú, ajustar patrón si es necesario)
    dni_pattern = r'\b[0-9]{8}\b' # 8 dígitos para DNI
    for match in re.finditer(dni_pattern, text):
        found_pii.append({'type': 'DNI', 'text': match.group(0), 'start': match.start(), 'end': match.end()})

    # 2. Spacy para NER
    doc = nlp(text)
    for ent in doc.ents:
        # Filtrar tipos de entidades que nos interesan como PII
        # if ent.label_ in ["PER", "LOC", "ORG",]: # PER: Personas, LOC: Lugares, ORG: Organizaciones, DATE: Fechas
        if ent.label_ in ["PER"]:            
            # Convierte a mayúsculas para hacer la comparación insensible a mayúsculas/minúsculas
            if ent.text.upper() not in EXCLUDED_LEGAL_TERMS:
                found_pii.append({'type': ent.label_, 'text': ent.text, 'start': ent.start_char, 'end': ent.end_char})
            else:
                print(f"DEBUG: Excluyendo término legal '{ent.text}' ({ent.label_}) de la anonimización.")

    return found_pii

In [14]:
pii_lista = find_pii(texto)
print(pii_lista)

DEBUG: Excluyendo término legal 'AUTOS' (PER) de la anonimización.
DEBUG: Excluyendo término legal 'VISTOS' (PER) de la anonimización.
[{'type': 'PHONE', 'text': ' 965403382', 'start': 3364, 'end': 3374}, {'type': 'PER', 'text': 'DELITO', 'start': 334, 'end': 340}, {'type': 'PER', 'text': 'Segundo Juzgado de Invest igación Preparatoria', 'start': 754, 'end': 800}, {'type': 'PER', 'text': 'Penal Corporativa de El Agustin', 'start': 891, 'end': 922}, {'type': 'PER', 'text': 'Decreto Legislativo N', 'start': 1085, 'end': 1106}, {'type': 'PER', 'text': 'Segundo \nJuzgado de Investigación Preparatoria', 'start': 1186, 'end': 1232}, {'type': 'PER', 'text': 'El Agustin', 'start': 1236, 'end': 1246}, {'type': 'PER', 'text': 'Audiencia Única', 'start': 1304, 'end': 1319}, {'type': 'PER', 'text': 'Juicio Inmediato', 'start': 1324, 'end': 1340}, {'type': 'PER', 'text': 'Juicio Oral', 'start': 1446, 'end': 1457}, {'type': 'PER', 'text': 'Dyland Caled Bracamonte Siguas', 'start': 1627, 'end': 1657}

In [15]:
def anonymize_text_with_x(original_text, found_pii):
    """
    Reemplaza las entidades PII en el texto original con el carácter 'X'.

    Args:
        original_text (str): El texto completo extraído del PDF.
        found_pii (list): Una lista de diccionarios, donde cada diccionario
                          representa una PII encontrada y tiene las claves
                          'start' (índice de inicio) y 'end' (índice de fin).
                          Ejemplo: [{'type': 'PERSON', 'text': 'Juan Pérez', 'start': 10, 'end': 20}]

    Returns:
        str: El texto con las PII reemplazadas por 'X'.
    """
    # Convertir el texto a una lista de caracteres para facilitar la modificación in-place
    # Esto es más eficiente que manipular cadenas repetidamente.
    text_chars = list(original_text)

    # Es crucial procesar las PII en orden inverso de sus posiciones
    # para que los reemplazos no afecten los índices de las PII aún no procesadas.
    # Alternativamente, se puede construir un nuevo texto.
    # Para este método de lista de caracteres, el orden no es tan crítico
    # si solo estamos reemplazando con 'X' de la misma longitud.
    # Sin embargo, si hubiera solapamientos, el orden inverso o un enfoque de "marcado"
    # sería mejor. Para simple reemplazo, podemos iterar directamente.

    for pii in found_pii:
        start_index = pii['start']
        end_index = pii['end']

        # Asegurarse de que los índices estén dentro de los límites del texto
        if 0 <= start_index < len(text_chars) and 0 <= end_index <= len(text_chars) and start_index < end_index:
            # Calcular la longitud de la PII
            length_of_pii = end_index - start_index
            # Reemplazar el segmento de texto con 'X'
            for i in range(start_index, end_index):
                text_chars[i] = 'X'
        else:
            print(f"Advertencia: Índices de PII inválidos o fuera de rango: {pii}. No se anonimizará.")

    # Unir la lista de caracteres de nuevo en una cadena
    anonymized_text = "".join(text_chars)
    return anonymized_text



In [16]:
texto_anonimizado = anonymize_text_with_x(texto, pii_lista)

In [17]:
print(texto_anonimizado)

JUZGADO PENAL UNIPERSONAL - SEDE NCPP EL AGUSTINO 
EXPEDIENTE : 00341-2024-1-3203-JR-PE-02 
JUEZ   : LOAYZA SANCHEZ RUTH KARINA 
ESPECIALISTA : CONDORI ACUÑA HERBERTH 
MINISTERIO PUBLICO : PRIMERA FISCALIA SEGUNDO DESPA CHO EL 
AGUSTINO  
REPRESENTANTE : SIGUAS ZEGARRA, MONICA DEL ROSARIO 
IMPUTADO : BRACAMONTE PEREZ, FREDDY SAMMY 
XXXXXX  : OMISIÓN DE ASISTENCIA FAMILIAR 
AGRAVIADO : BRACAMONTE SIGUAS, DYLAND CALEB 
 
RESOLUCIÓN NÚMERO UNO    
El Agustino, veintiséis de diciembre  
Del dos mil veinticuatro. - 
    
   AUTOS Y VISTOS : AVOCÁNDOSE  la suscrita conforme a 
lo dispuesto en la Resolución Administrativa N°152- 2022-P-CSJLE-PJ; y al 
oficio con ingreso número 202583-2024 Téngase por r ecibido el expediente 
principal proveniente del XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX 
de El Agustino, conteniendo la acusación presentada  por la Primera 
fiscalía provincial XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX o – Segundo despacho 
de Investigación, y, ATENDIENDO :    
    
PRIMERO .- Q